<a href="https://colab.research.google.com/github/Saudii81/fcc_cat_dog_image_classifier/blob/main/FCC_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-09-25 11:45:37--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  32.6MB/s    in 0.8s    

2025-09-25 11:45:39 (32.6 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Step 1: Filter users with >=200 ratings and books with >=100 ratings
user_counts = df_ratings['user'].value_counts()
book_counts = df_ratings['isbn'].value_counts()

df_ratings_filtered = df_ratings[
    df_ratings['user'].isin(user_counts[user_counts >= 200].index)
]
df_ratings_filtered = df_ratings_filtered[
    df_ratings_filtered['isbn'].isin(book_counts[book_counts >= 100].index)
]

# Step 2: Create user-item matrix (sparse for efficiency)
user_item_matrix = df_ratings_filtered.pivot_table(
    index='user', columns='isbn', values='rating'
).fillna(0)

sparse_matrix = csr_matrix(user_item_matrix.values)

# Step 3: Train Nearest Neighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(sparse_matrix.T)

# Step 4: ISBN ↔ Title mappings
isbn_to_title = df_books.set_index('isbn')['title'].to_dict()
title_to_isbn = {v: k for k, v in isbn_to_title.items()}

# Step 5: Define recommendation function
def get_recommends(book=""):
    if book not in title_to_isbn:
        return [book, []]

    isbn = title_to_isbn[book]
    if isbn not in user_item_matrix.columns:
        return [book, []]

    book_vector = user_item_matrix[isbn].values.reshape(1, -1)
    distances, indices = model.kneighbors(book_vector, n_neighbors=6)

    recs = []
    for dist, idx in zip(distances.flatten()[1:], indices.flatten()[1:]):
        rec_isbn = user_item_matrix.columns[idx]
        rec_title = isbn_to_title.get(rec_isbn, "Unknown")
        recs.append([rec_title, float(dist)])

    return [book, recs]


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book=""):
    if book not in title_to_isbn:
        return [book, []]

    isbn = title_to_isbn[book]
    if isbn not in user_item_matrix.columns:
        return [book, []]

    book_vector = user_item_matrix[isbn].values.reshape(1, -1)
    distances, indices = model.kneighbors(book_vector, n_neighbors=6)

    recs = []
    for dist, idx in zip(distances.flatten()[1:], indices.flatten()[1:]):
        rec_isbn = user_item_matrix.columns[idx]
        rec_title = isbn_to_title.get(rec_isbn, "Unknown")
        recs.append([rec_title, float(dist)])

    return [book, recs]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.7234864234924316], ['I Know This Much Is True', 0.7677075266838074], ['The Surgeon', 0.7699410915374756], ['The Weight of Water', 0.7708583474159241], ["I'll Be Seeing You", 0.8016210794448853]]]
You haven't passed yet. Keep trying!
